# Samples 04-kernel-arguments-chat

Source : <https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/04-kernel-arguments-chat.ipynb>

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

Installed Packages Microsoft.SemanticKernel, 1.11.1

## Instantiate the kernel


In [4]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

Console.WriteLine($"Use Azure Open AI : {useAzureOpenAI}");
Console.WriteLine($"Model : {model}");
Console.WriteLine($"Azure EndPoint : {azureEndpoint}");
// Console.WriteLine($"API Key : {apiKey}");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Use Azure Open AI : True
Model : gpt-35-turbo-16k
Azure EndPoint : https://oai-ygo74-switzerland.openai.azure.com/


In [5]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

In [6]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

In [7]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

In [8]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

In [9]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help you with book suggestions. What genre or type of book are you interested in?



In [10]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;

    // Show the response
    Console.WriteLine(result);
};

In [11]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Sure! I would recommend "The Peloponnesian War" by Thucydides. It is a classic historical account of the war between Athens and Sparta, providing valuable insights into ancient Greek history.



In [12]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Peloponnesian War," you will learn about the causes and events of the war, the strategies and tactics employed by both sides, the political and social dynamics of ancient Athens and Sparta, the role of other Greek city-states in the conflict, and the long-term consequences of the war on Greek history.



In [13]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: I think many people find the political and social dynamics of ancient Athens and Sparta to be particularly interesting. These two city-states had very different forms of government and societal structures, and understanding how they interacted and clashed during the war can provide valuable insights into the ancient Greek world.



In [14]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: Certainly! Here are a few more books you could consider reading about ancient Greek history:

1. "The Histories" by Herodotus - This is another classic work that provides a comprehensive account of ancient Greek history, including the Persian Wars.

2. "The Rise and Fall of Athens: Nine Greek Lives" by Plutarch - This book offers biographies of nine influential figures from ancient Athens, providing insights into the political and cultural developments of the city-state.

3. "The Spartans: The World of the Warrior-Heroes of Ancient Greece" by Paul Cartledge - This book focuses specifically on the history and culture of Sparta, offering a deeper understanding of the Spartan society and its role in ancient Greece.

4. "The Greco-Persian Wars" by Peter Green - This book delves into the conflicts between the Greek city-states and the Persian Empire, exploring the political, military, and cultural aspec

In [15]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help you with book suggestions. What genre or type of book are you interested in?

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Sure! I would recommend "The Peloponnesian War" by Thucydides. It is a classic historical account of the war between Athens and Sparta, providing valuable insights into ancient Greek history.

User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Peloponnesian War," you will learn about the causes and events of the war, the strategies and tactics employed by both sides, the political and social dynamics of ancient Athens and Sparta, the role of other Greek city-states in the conflict, and the long-term consequences of the war on Greek history.

User: Which topic from the ones you listed do you think most people find interesting?
AI: I think many people find the political and social dynamics of ancient